# Livneh Average Tmax

In [1]:
# imports
import numpy
import Nio
import Ngl
from IPython import display
import netCDF4 as nc
from datetime import date
from datetime import timedelta
import xarray as xr
import xesmf as xe

In [2]:
# reading in cpc
fn = '/cw3e/mead/projects/cwp103/scratch/ldehaan/CPC/tmax.2017.nc'
liv = xr.open_dataset(fn)
liv['tmax'][35:46]

<xarray.DataArray 'tmax' (time: 11, lat: 360, lon: 720)>
[2851200 values with dtype=float32]
Coordinates:
  * lat      (lat) float32 89.75 89.25 88.75 88.25 ... -88.75 -89.25 -89.75
  * lon      (lon) float32 0.25 0.75 1.25 1.75 2.25 ... 358.2 358.8 359.2 359.8
  * time     (time) datetime64[ns] 2017-02-05 2017-02-06 ... 2017-02-15
Attributes: (12/13)
    units:         degC
    var_desc:      Maximum Temperature
    level_desc:    Surface
    statistic:     Maximum
    parent_stat:   Other
    long_name:     Daily Maximum Temperature
    ...            ...
    valid_range:   [-90.  50.]
    avg_period:    0000-00-01 00:00:00
    dataset:       CPC Global Temperature
    comments:      GTS data and is gridded using the Shepard Algorithm
    max_period:    6z to 6z
    actual_range:  [-56.827217  55.789146]

In [3]:
# averaging
sum_arr = liv['tmax'][35].data
for i in numpy.arange(36, 46): # before time shift 36, 46
    sum_arr += liv['tmax'][i].data
avg = sum_arr / 11 #13

# for reference
numpy.unique(avg), numpy.unique(sum_arr)

(array([-39.154995, -38.238155, -37.705044, ...,        nan,        nan,
               nan], dtype=float32),
 array([-430.70493, -420.6197 , -414.7555 , ...,        nan,        nan,
               nan], dtype=float32))

In [9]:
# janky way to find average using wrf coords, subtracting livneh avg/pt by random 
# wrf to create nans then adding livneh avg/pt back so we can remove pts not in common
wrf_05 = xr.open_dataset('./wrf_05_cpc-Copy1.nc')
wf = wrf_05['maxes'].data
wf[wf == 0.0] = 'nan'
sub = numpy.subtract(avg, wf)
avg2 = numpy.add(wf, sub)
numpy.unique(wf), numpy.unique(sub), numpy.unique(avg2)
sum(~numpy.isnan(avg2.flatten()))

2041

In [5]:
# convert dif_arr to masked array to overlook NaNs
import numpy.ma as ma
m_out = ma.masked_invalid(avg)
m_out

masked_array(
  data=[[--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        ...,
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --]],
  mask=[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],
  fill_value=1e+20,
  dtype=float32)

In [6]:
# numerical avg stat over entire region
numpy.mean(m_out.flatten()), min(numpy.unique(m_out)), max(numpy.unique(m_out))

(5.90780154180118, -39.154995, 43.04112)

# borrow RHS plot end coords
fn = '/cw3e/mead/projects/cwp103/production/output/2017020500/cf/NG-GR-NQ-BD/wrfcf_d02_2017-02-05_22_00_00.nc'
da_input = xr.open_dataset(fn)

wks_type2 = "png"
wks2 = Ngl.open_wks(wks_type2,"17_cpc_avg")

# defining variables
temp = m_out
lat = liv['lat'].data.flatten()
lon = liv['lon'].data.flatten()
lat2 = da_input['lat'].data.flatten()
lon2 = da_input['lon'].data.flatten()
tmax_nlat = len(lat)
tmax_nlon = len(lon)

# XY of overlay plot
resources = Ngl.Resources()
resources.sfXCStartV = min(lon)
resources.sfXCEndV   = max(lon)
resources.sfYCStartV = min(lat)
resources.sfYCEndV   = max(lat)

# fill missing values as needed
if hasattr(temp,"_FillValue"):
    resources.sfMissingValueV = temp._FillValue

# plot styling
resources.cnLevelSelectionMode = "ManualLevels"
resources.cnMinLevelValF       = -20
resources.cnMaxLevelValF       = 32
resources.cnLevelSpacingF      = 4
resources.tiXAxisString = "~F25~Longitude"
resources.tiYAxisString = "~F25~Latitude"
resources.cnFillOn              = True    
resources.cnLineLabelsOn        = False  
resources.cnInfoLabelOn         = False  
resources.nglSpreadColorEnd     = -2
resources.mpProjection = "LambertEqualArea" 
resources.mpCenterLonF = (lon[tmax_nlon-1] + lon[0])/2
resources.mpCenterLatF = (lat[tmax_nlat-1] + lat[0])/2
resources.mpLimitMode = "LatLon"
resources.mpMinLonF   = min(lon)
resources.mpMaxLonF   = max(lon)
resources.mpMinLatF   = min(lat)
resources.mpMaxLatF   = max(lat)
resources.mpPerimOn   = True
resources.cnLinesOn   = False
resources.mpOutlineBoundarySets = "GeophysicalAndUSStates"
resources.pmTickMarkDisplayMode = "Never"
resources.tiMainString = "~F26~CPC Tmax Average, 2017-02"
resources.lbOrientation  = "Horizontal"
resources.lbTitleString  = "~F25~Max Temp (C)"

# plot
map = Ngl.contour_map(wks2,temp,resources)

# clean up
del map
del resources
Ngl.end()

# display plot
display.Image("./17_cpc_avg.png")